In [7]:
import pandas as pd
import numpy as np
 
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

from utils_data import Vocabulary, clean_words

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [8]:
from datasets import load_dataset

# data_en = load_dataset('head_qa', 'en')
data_es = load_dataset('head_qa', 'es' )
data_es

Dataset head_qa downloaded and prepared to C:\Users\CLAUDIA\.cache\huggingface\datasets\head_qa\es\1.1.0\d6803d1e84273cdc4a2cf3c5102945d166555f47b299ecbc5266d582f408f8e2. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['name', 'year', 'category', 'qid', 'qtext', 'ra', 'image', 'answers'],
        num_rows: 2657
    })
    test: Dataset({
        features: ['name', 'year', 'category', 'qid', 'qtext', 'ra', 'image', 'answers'],
        num_rows: 2742
    })
    validation: Dataset({
        features: ['name', 'year', 'category', 'qid', 'qtext', 'ra', 'image', 'answers'],
        num_rows: 1366
    })
})

In [9]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [10]:
for d in training:
    print(d)
    break;

{'name': 'Cuaderno_2013_1_B', 'year': '2013', 'category': 'biology', 'qid': 1, 'qtext': 'Los potenciales postsinápticos excitadores:', 'ra': 3, 'image': '', 'answers': [{'aid': 1, 'atext': 'Son de tipo todo o nada.'}, {'aid': 2, 'atext': 'Son hiperpolarizantes.'}, {'aid': 3, 'atext': 'Se pueden sumar.'}, {'aid': 4, 'atext': 'Se propagan a largas distancias.'}, {'aid': 5, 'atext': 'Presentan un periodo refractario.'}]}


In [13]:
#!python -m spacy download es_core_news_sm

In [14]:
import spacy
nlp = spacy.load("es_core_news_sm")

vocabulary = Vocabulary()

In [17]:
def parse_training(training):
    train = []       
    for sample in training:
        training_sample = {}
        qtext = sample['qtext']
        answers = sample['answers']        
        q = nlp(qtext)
        tok_qtext = [token.text for token in q]
        right_answer = sample['ra']        
        for answer in answers:
            aid, atext = answer['aid'], answer['atext']
            a = nlp(atext)
            tok_atext = [token.text for token in a]
            instance_x = tok_qtext + ['SEP'] + tok_atext    
            instance_y = 1 if right_answer == aid else 0
            training_sample['question'] = qtext
            training_sample['answer'] = atext
            training_sample['label'] = instance_y
            training_sample['sample_tok'] = instance_x
            training_sample['category'] = sample['category']
            train.append(training_sample)
            print(training_sample)
        break

In [18]:
parse_training(training)

{'question': 'Los potenciales postsinápticos excitadores:', 'answer': 'Son de tipo todo o nada.', 'label': 0, 'sample_tok': ['Los', 'potenciales', 'postsinápticos', 'excitadores', ':', 'SEP', 'Son', 'de', 'tipo', 'todo', 'o', 'nada', '.'], 'category': 'biology'}
{'question': 'Los potenciales postsinápticos excitadores:', 'answer': 'Son hiperpolarizantes.', 'label': 0, 'sample_tok': ['Los', 'potenciales', 'postsinápticos', 'excitadores', ':', 'SEP', 'Son', 'hiperpolarizantes', '.'], 'category': 'biology'}
{'question': 'Los potenciales postsinápticos excitadores:', 'answer': 'Se pueden sumar.', 'label': 1, 'sample_tok': ['Los', 'potenciales', 'postsinápticos', 'excitadores', ':', 'SEP', 'Se', 'pueden', 'sumar', '.'], 'category': 'biology'}
{'question': 'Los potenciales postsinápticos excitadores:', 'answer': 'Se propagan a largas distancias.', 'label': 0, 'sample_tok': ['Los', 'potenciales', 'postsinápticos', 'excitadores', ':', 'SEP', 'Se', 'propagan', 'a', 'largas', 'distancias', '.'],

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, x_size, n_classes): 
        super(LogisticRegression, self).__init__()             
        self.linear = nn.Linear(x_size, n_classes)
        
    def forward(self, x):
        return self.linear(x)